# Filtering Ofcom Wireless Telegraphy Register

The Wireless Telegraphy Register (WTR) is a publicly available database containing all of the UK's licencing information. The WTR contains information such as licence number, issue date, coordinates and licence holder among many others. The scope of the WTR is much larger than is required to visualise released spectrum in for Wireless Broad Access and will require extensive filtering and conditional formatting.

The WTR contains every licence, over all of the UK's spectrum. Filters must be developed to isolate mid-band licences used exclusively for WBA and must exclude extraneous information such as antenna specifications and device location.

## WTR Complications

This notebook will not compile with the source WTR.csv file which is available on the Ofcom website. In order to compile this notebook on a local machine, be sure to download the source data and place it in the project folder.

Unlike the RRL, the required data exists in a single CSV file, however, there aren't any Ofcom classifications for WBA licences which means that filter criteria need developed very carefully to ensure all non-WBA licences are excluded.

> After further investigating, the WTR does not contain complete information of the UKs WBA allocation. The appropriate information can be found in UK Spectrum Map table.

In [69]:
import pandas as pd
import numpy as np

wtr = pd.read_csv('WTR.csv')

# Remove unnecessary columns
wtr = wtr[['Licence Number', 
    'Frequency (Hz)', 
    'Station Type', 
    'Channel Width (Hz)', 
    'Product Description']]

# Convert Hz to MHz
wtr['Frequency (Hz)'] = (wtr['Frequency (Hz)'] / 1000000).round(5)
wtr['Channel Width (Hz)'] = (wtr['Channel Width (Hz)'] / 1000000).round(5)

# Change column names
wtr.rename(columns={'Frequency (Hz)' : 'FREQUENCY',
    'Channel Width (Hz)' : 'BW',
    'Licence Number' : 'LICENCE_NO',
    'Station Type' : 'STATION_TYPE',
    'Product Description' : 'CATEGORY'}, inplace=True)

# Isolate mid-band frequencies
wtr = wtr[wtr['FREQUENCY'] < 7000]
wtr = wtr[wtr['FREQUENCY'] > 700]
wtr.sort_values(by=['FREQUENCY','LICENCE_NO'], inplace=True)
wtr.reset_index(drop=True, inplace=True)

# Save dataset to CSV
wtr.to_csv('Ofcom Datasets/spectrumLicences.csv', index=False)
wtr

,LICENCE_NO,FREQUENCY,STATION_TYPE,BW,CATEGORY
0,0926980/1,1164.0,T,NaN,GNSS Repeater
1,0926980/1,1164.0,T,NaN,GNSS Repeater
2,0926980/1,1164.0,T,NaN,GNSS Repeater
3,0929062/1,1164.0,T,NaN,GNSS Repeater
4,0929080/1,1164.0,T,NaN,GNSS Repeater
...,...,...,...,...,...
8716,1245331/3,6980.0,R,30.0,Fixed Links
8717,1256067/1,6980.0,R,30.0,Fixed Links
8718,1256067/1,6980.0,T,30.0,Fixed Links
8719,1270272/1,6980.0,R,30.0,Fixed Links


## UK Spectrum Map Dataset

Ofcom provides a spectrum map on their website which has access to a database of spectrum licences which include WBA licences that are not found in the WTR. The spectrum source data has been downloaded but requires filtering to remove all licences which are not WBA.

> NOTE: There are two WBA licence categories in the spectrum map, 'Mobile and Wireless Broadband' (capital B) has been excluded due extensive overlap.

### Acquiring Spectrum Map Source Data

The Spectrum Map source can be found at:

http://static.ofcom.org.uk/static/spectrum/map.html

Scroll down to the bottom left of the page and click on the 'Download Source Data' prompt. The page will then load a JSON file containing all relevant licences. Be sure to remove the 'terms_of_use' and 'date_updated' attributes so that JSON to CSV conversion will not be negatively impacted.

The resulting JSON file can be converted to CSV at:

https://www.convertcsv.com/json-to-csv.htm

Be sure to name the resulting csv file 'spectrumMap.csv' and place it in the project folder. Failing to do so will prevent the notebook from compiling correctly.

In [89]:
import numpy as np

spectrumMap = pd.read_csv('spectrumMap.csv')

# Isolate wireless broadband entries
spectrumMap = spectrumMap[spectrumMap['bands/s'] == 'Mobile and Wireless broadband']
spectrumMap = spectrumMap[spectrumMap['bands/s'] == 'Mobile and Wireless broadband']

# Convert Hz to MHz
spectrumMap['bands/lf'] = (spectrumMap['bands/lf'] / 1000000).round(5)
spectrumMap['bands/uf'] = (spectrumMap['bands/uf'] / 1000000).round(5)
spectrumMap['bw'] = spectrumMap['bands/uf'] - spectrumMap['bands/lf']

# Isolate midband licences
spectrumMap = spectrumMap[spectrumMap['bands/lf'] <= 7000]
spectrumMap = spectrumMap[spectrumMap['bands/lf'] >= 700]

# Create 100 MHz categories + GHz classifications
spectrumMap['range'] = (np.trunc((spectrumMap['bands/lf'] / 100))*100)
spectrumMap['GHz'] = (spectrumMap['range'] / 1000).round()

# Create freq range column
# spectrumMap['freq_range'] = str(spectrumMap['bands/lf']) + str(spectrumMap['bands/uf'])

spectrumMap.sort_values(by=['bands/lf'])
spectrumMap.reset_index(drop=True, inplace=True)
spectrumMap.to_csv('Ofcom Datasets/spectrumMap.csv')
spectrumMap

,bands/lf,bands/uf,bands/s,bands/u,bands/v,bw,range,GHz
0,703.0,713.0,Mobile and Wireless broadband,Spectrum Access Telefonica (Uplink),1,10.0,700.0,1.0
1,713.0,723.0,Mobile and Wireless broadband,Spectrum Access Hutchison 3G (Uplink),1,10.0,700.0,1.0
2,723.0,733.0,Mobile and Wireless broadband,Spectrum Access EE (Uplink),1,10.0,700.0,1.0
3,758.0,768.0,Mobile and Wireless broadband,Spectrum Access Telefonica (Downlink),1,10.0,700.0,1.0
4,778.0,788.0,Mobile and Wireless broadband,Spectrum Access EE (Downlink),1,10.0,700.0,1.0
...,...,...,...,...,...,...,...,...
75,3760.0,3800.0,Mobile and Wireless broadband,Spectrum Access Telefonica,1,40.0,3700.0,4.0
76,3800.0,4200.0,Mobile and Wireless broadband,Shared Access (Low Power),1,400.0,3800.0,4.0
77,3800.0,4200.0,Mobile and Wireless broadband,Shared Access (Medium Power),1,400.0,3800.0,4.0
78,3925.0,4009.0,Mobile and Wireless broadband,Spectrum Access UK Broadband,1,84.0,3900.0,4.0


### Unallocated function

While the data in the tables are useful, it often doesn't easily represent the utilisation of certain frequency ranges. It is up to the viewer to infer whether or not spectrum is being used for WBA therefore, it is important to calculate an figure for the amount of spectrum that is unallocated for a given dataset.

In [81]:
# Unallocated requires a very specific dataset.
# Columns must contain: lower freq, higher freq, bw, range
# This function will adhere to column names for the UK Spectrum map dataset
def calculateUnallocated (dataset, i):
    # Select a single frequency graduation
    graduated = dataset[dataset['range'] == i]

    # Calculate sum
    sum = graduated.sum(axis=0).to_frame().T
    graduated = pd.concat([graduated, sum])

    # Remove junk values
    graduated.iloc[-1, graduated.columns.get_loc('bands/lf')] = np.nan
    graduated.iloc[-1, graduated.columns.get_loc('bands/uf')] = np.nan
    graduated.iloc[-1, graduated.columns.get_loc('bands/v')] = np.nan
    graduated.iloc[-1, graduated.columns.get_loc('bands/u')] = np.nan
    graduated.iloc[-1, graduated.columns.get_loc('range')] = np.nan

    # Calculate unallocated spectrum from usage
    usageBW = graduated.iloc[-1, graduated.columns.get_loc('bw')]
    freeBW = 100 - usageBW
    if freeBW < 0:
        freeBW = 0 
    graduated.iloc[-1, graduated.columns.get_loc('bw')] = freeBW

    # Update classification and range
    graduated.iloc[-1, graduated.columns.get_loc('bands/s')] = 'Unallocated'
    graduated.iloc[-1, graduated.columns.get_loc('bands/u')] = 'Unallocated'
    graduated.iloc[-1, graduated.columns.get_loc('bands/lf')] = 'Unallocated'
    graduated.iloc[-1, graduated.columns.get_loc('range')] = float(i)

    return graduated

def unallocated (dataset):

    # Loop through frequency graduations (100 MHz)
    # between 0 - 7000 MHz
    for i in range(0, 7100, 100):

        if i == 0:
            
            usage = calculateUnallocated(dataset, i)
        
        else:

            nextUsage = calculateUnallocated (dataset, i)
            usage = pd.concat([usage, nextUsage])

    # Reset index and save dataset
    usage.reset_index(drop=True, inplace=True)  
    usage.to_csv('Ofcom Datasets/UKSpecMapUsage.csv')     
    return usage

UKSpectrumMap = unallocated(spectrumMap)
UKSpectrumMap

        

,bands/lf,bands/uf,bands/s,bands/u,bands/v,bw,range
0,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,0.0
1,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,100.0
2,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,200.0
3,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,300.0
4,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,400.0
...,...,...,...,...,...,...,...
146,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,6600.0
147,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,6700.0
148,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,6800.0
149,Unallocated,NaN,Unallocated,Unallocated,NaN,100.0,6900.0


## User Generated Dataframe

There is a user generated dataset containing 100 MHz frequency summaries for mid-band wireless broadband licences below 5 GHz.

> NOTE: Using the transpose functions breaks indexing, ignore strange index name

In [72]:
userGen = pd.read_csv('Ofcom Datasets/userGeneratedDataset.csv')
userGen = userGen.set_index('Frequency').transpose()
userGen.reset_index(inplace=True)

# Rename Columns
userGen = userGen.rename(columns={'index' : 'FREQUENCY',
    'Usage (MHz)' : 'BW', 
    'Uplink Usage (MHz)' : 'UP_BW',
    'Downlink Usage (MHz)' : 'LW_BW',
    'Link Usage (MHz)' : 'L_BW'}, inplace=False)

#userGen['BW'] = userGen['BW'].fillna(0)
userGen['FREE'] = 100 - userGen['BW']

# Separate Uplink, Downlink, Total and Free
uplink = userGen[['FREQUENCY','UP_BW','Uplinks']]
uplink['TYPE'] = 'UPLINK'
uplink.rename(columns={'UP_BW':'BW','Uplinks':'Licences'}, inplace=True)

downlink = userGen[['FREQUENCY', 'LW_BW', 'Downlinks']]
downlink['TYPE'] = 'DOWNLINK'
downlink.rename(columns={'LW_BW':'BW','Downlinks':'Licences'}, inplace=True)

link = userGen[['FREQUENCY', 'L_BW', 'Links']]
link['TYPE'] = 'LINK'
link.rename(columns={'L_BW':'BW','Links':'Licences'}, inplace=True)

#total = userGen[['FREQUENCY', 'BW', 'Complete Licences']]
#total['TYPE'] = 'TOTAL'
#total.rename(columns={'Complete Licences':'Licences'}, inplace=True)

free = userGen[['FREQUENCY', 'FREE']]
free['Licences'] = 1
free['TYPE'] = 'UNALLOCATED'
free.rename(columns={'FREE':'BW'}, inplace=True)

userGenFreq = pd.concat([uplink, downlink, link, free], axis=0)
userGenFreq = userGenFreq.astype({'FREQUENCY' : 'int64'})
userGenFreq = userGenFreq.sort_values(by=['FREQUENCY'], inplace=False)

userGenFreq.to_csv('Ofcom Datasets/userGenFreqList.csv', index=False)
userGenFreq

C:\Users\Nav\AppData\Local\Temp\ipykernel_11908\1481661842.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Nav\AppData\Local\Temp\ipykernel_11908\1481661842.py:18: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Nav\AppData\Local\Temp\ipykernel_11908\1481661842.py:21: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

Frequency,FREQUENCY,BW,Licences,TYPE
0,700,30.0,3.0,UPLINK
0,700,0.0,0.0,LINK
0,700,36.0,1.0,UNALLOCATED
0,700,34.0,5.0,DOWNLINK
1,800,49.9,7.0,UPLINK
...,...,...,...,...
42,4900,NaN,NaN,UPLINK
43,5000,NaN,NaN,DOWNLINK
43,5000,NaN,NaN,UPLINK
43,5000,NaN,NaN,LINK


# DATA VISUALISATION

## User Generated Frequency List

In [73]:
import plotly_express as px

fig = px.treemap(userGenFreq, path=[px.Constant('1 - 5GHz'), 'FREQUENCY', 'TYPE'], 
    values='BW', color='TYPE',
    color_discrete_map={'(?)':'lightgrey', 'UNALLOCATED' : 'grey'})
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/userGenFreqTreemap.html')
fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



## UK Spectrum Treemap Dataset

The UK Spectrum treemap dataset contains a more granular visualisation of the wireless broadband access licences in the UK.

> Displaying the entire dataset in treemap is not very useful because of the sheer number of 100MHz graduations. Use subplots to break dataset into GHz graduations.

In [139]:
UKSpectrumMap1 = UKSpectrumMap[UKSpectrumMap['range'] <= 1000]
UKSpectrumMap2 = UKSpectrumMap[(UKSpectrumMap['range'] <= 2000) & (UKSpectrumMap['range'] >= 1000)]
UKSpectrumMap3 = UKSpectrumMap[(UKSpectrumMap['range'] <= 3000) & (UKSpectrumMap['range'] >= 2000)]
UKSpectrumMap4 = UKSpectrumMap[(UKSpectrumMap['range'] <= 4000) & (UKSpectrumMap['range'] >= 3000)]
UKSpectrumMap5 = UKSpectrumMap[(UKSpectrumMap['range'] <= 5000) & (UKSpectrumMap['range'] >= 4000)]
UKSpectrumMap6 = UKSpectrumMap[(UKSpectrumMap['range'] <= 6000) & (UKSpectrumMap['range'] >= 5000)]
UKSpectrumMap7 = UKSpectrumMap[(UKSpectrumMap['range'] <= 7000) & (UKSpectrumMap['range'] >= 6000)]

# 1 GHz Treemap
fig = px.treemap(UKSpectrumMap1, path=[px.Constant('1 GHz'),
    'range',
    'bands/u', 
    'bands/lf'], 
    values='bw',
    color='bands/lf',
    color_discrete_map={'(?)':'lightgrey', 'Unallocated' : 'grey'})

fig.update_traces(root_color="lightgrey")
fig.update_layout(margin_autoexpand=True)
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap1.html')
treemap1 = fig

# 2 GHz Treemap
fig = px.treemap(UKSpectrumMap2, path=[px.Constant('2 GHz'),
    'range',
    'bands/u', 
    'bands/lf'], 
    values='bw',
    color='bands/lf',
    color_discrete_map={'(?)':'lightgrey', 'Unallocated' : 'grey'})

fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap2.html')
treemap2 = fig

# 3 GHz Treemap
fig = px.treemap(UKSpectrumMap3, path=[px.Constant('3 GHz'),
    'range',
    'bands/u', 
    'bands/lf'], 
    values='bw',
    color='bands/lf',
    color_discrete_map={'(?)':'lightgrey', 'Unallocated' : 'grey'})

fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap3.html')
treemap3 = fig


# 4 GHz Treemap
fig = px.treemap(UKSpectrumMap4, path=[px.Constant('4 GHz'),
    'range',
    'bands/u', 
    'bands/lf'], 
    values='bw',
    color='bands/lf',
    color_discrete_map={'(?)':'lightgrey', 'Unallocated' : 'grey'})

fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap4.html')
treemap4 = fig


# 5 GHz Treemap
fig = px.treemap(UKSpectrumMap5, path=[px.Constant('5 GHz'),
    'range',
    'bands/u', 
    'bands/lf'], 
    values='bw',
    color='bands/lf',
    color_discrete_map={'(?)':'lightgrey', 'Unallocated' : 'grey'})

fig.update_traces(root_color="lightgrey")
#fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap5.html')
treemap5 = fig


# 6 GHz Treemap
fig = px.treemap(UKSpectrumMap6, path=[px.Constant('6 GHz'),
    'range',
    'bands/u', 
    'bands/lf'], 
    values='bw',
    color='bands/lf',
    color_discrete_map={'(?)':'lightgrey', 'Unallocated' : 'grey'})

fig.update_traces(root_color="lightgrey")
#fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap6.html')
treemap6 = fig


# 7 GHz Treemap
fig = px.treemap(UKSpectrumMap7, path=[px.Constant('7 GHz'),
    'range',
    'bands/u', 
    'bands/lf'], 
    values='bw',
    color='bands/lf',
    color_discrete_map={'(?)':'lightgrey', 'Unallocated' : 'grey'})

fig.update_traces(root_color="lightgrey")
#fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.update_traces(tiling_packing='slice-dice')
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap7.html')
treemap7 = fig

fig.show()

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Nav\AppData\Local\Programs\Python\Python310\lib

In [153]:
from plotly.subplots import make_subplots

fig = make_subplots(
    cols = 1, rows = 7,
    subplot_titles = ('WBA Licences up to 1GHz', 
        'WBA Licences between 1 and 2GHz',
        'WBA Licences between 2 and 3GHz',
        'WBA Licences between 3 and 4GHz',
        'WBA Licences between 4 and 5GHz',
        'WBA Licences between 5 and 6GHz',
        'WBA Licences between 6 and 7GHz'
    ),
    specs = [[{'type': 'treemap'}],
        [{'type': 'treemap'}],
        [{'type': 'treemap'}],
        [{'type': 'treemap'}],
        [{'type': 'treemap'}],
        [{'type': 'treemap'}],
        [{'type': 'treemap'}]
    ], vertical_spacing=0.02
)

fig.add_trace(treemap1.data[0],row=1,col=1)
fig.add_trace(treemap2.data[0],row=2,col=1)
fig.add_trace(treemap3.data[0],row=3,col=1)
fig.add_trace(treemap4.data[0],row=4,col=1)
fig.add_trace(treemap5.data[0],row=5,col=1)
fig.add_trace(treemap6.data[0],row=6,col=1)
fig.add_trace(treemap7.data[0],row=7,col=1)
fig.update_layout(height=4000)
fig.write_html('Ofcom Treemaps/UKSpectrumMapTreemap0.html')
fig.show()
